In [1]:
# %pip install transformers datasets accelerate

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
from datasets import Dataset
from pathlib import Path
import torch
from tqdm.notebook import tqdm
import re

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
!wget -nc https://raw.githubusercontent.com/v-goncharenko/madmo-adv/55d929befa12370fc18109f5333f7cf000ea27ce/homeworks/onegin.txt

File ‘onegin.txt’ already there; not retrieving.



In [5]:
train_path = "onegin.txt"
model_name = "ai-forever/rugpt3small_based_on_gpt2"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [7]:
with open("onegin.txt", "r") as iofile:
    text = iofile.readlines()

TEXT_START = 0
TEXT_END = -1
text = text[TEXT_START:TEXT_END]

In [8]:
def split_into_sonnets(text):
    sonnets = []
    cur_sonet = ""
    for line in text:
        line = line.strip()
        if re.match(r"^\b[IVXLCDM]+\b$", line):
            if cur_sonet:
                sonnets.append(cur_sonet)
                cur_sonet = ""
        elif line:
            cur_sonet += line + "\n"
    if cur_sonet:
        sonnets.append(cur_sonet)
    return sonnets

In [9]:
sonnets = split_into_sonnets(text)
print(sonnets[0], sep="\n")
print(sonnets[1], sep="\n")

«Мой дядя самых честных правил,
Когда не в шутку занемог,
Он уважать себя заставил
И лучше выдумать не мог.
Его пример другим наука;
Но, боже мой, какая скука
С больным сидеть и день и ночь,
Не отходя ни шагу прочь!
Какое низкое коварство
Полуживого забавлять,
Ему подушки поправлять,
Печально подносить лекарство,
Вздыхать и думать про себя:
Когда же черт возьмет тебя!»

Так думал молодой повеса,
Летя в пыли на почтовых,
Всевышней волею Зевеса
Наследник всех своих родных. —
Друзья Людмилы и Руслана!
С героем моего романа
Без предисловий, сей же час
Позвольте познакомить вас:
Онегин, добрый мой приятель,
Родился на брегах Невы,
Где, может быть, родились вы
Или блистали, мой читатель;
Там некогда гулял и я:
Но вреден север для меня



In [10]:
dataset = Dataset.from_dict({"text": sonnets})

max_length = 128
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=max_length)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Map:   0%|          | 0/358 [00:00<?, ? examples/s]

In [17]:
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50264, bias=False)
)

In [18]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
training_args = TrainingArguments(
    output_dir="./onegin_rugpt3_model",
    overwrite_output_dir=True,
    num_train_epochs=30,
    per_device_train_batch_size=32,
    save_steps=100,
    save_total_limit=1,
    logging_steps=10,
    logging_dir="./logs",
    report_to="none",
    disable_tqdm=False,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
)

In [20]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
10,4.147500
20,3.661300
30,3.369600
40,3.113700
50,2.918900
60,2.740400
70,2.572600
80,2.443400
90,2.319400
100,2.209700


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=180, training_loss=2.484167046017117, metrics={'train_runtime': 255.9125, 'train_samples_per_second': 41.967, 'train_steps_per_second': 0.703, 'total_flos': 701569105920000.0, 'train_loss': 2.484167046017117, 'epoch': 30.0})

In [21]:
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, truncation=True, device=0 if torch.cuda.is_available() else -1)

prompt = "Я вас любил"
outputs = generator(prompt, max_length=100, num_return_sequences=1, do_sample=True, top_k=50, top_p=0.95)

print(outputs[0]["generated_text"])

Device set to use cuda:0


Я вас любил, Таня, но забыть…
– Ах, милый… уж больно…
К чему так много?
К тому, что пора нам расстаться;
Я должен ехать;
Я вас пророните
Сердечной дозой.
Куда, милый мой? завтра же
Возьму в Москве у Тани больничную койку.
Онегин, верный мой,
Онегин… проститься с женой собрался.
Куда?.. где? к ней?


In [22]:
trainer.save_model("./onegin_rugpt3_model")
tokenizer.save_pretrained("./onegin_rugpt3_model")

('./onegin_rugpt3_model/tokenizer_config.json',
 './onegin_rugpt3_model/special_tokens_map.json',
 './onegin_rugpt3_model/vocab.json',
 './onegin_rugpt3_model/merges.txt',
 './onegin_rugpt3_model/added_tokens.json',
 './onegin_rugpt3_model/tokenizer.json')

In [27]:
Path('/kaggle/working/onegin_model_archive.zip').unlink()